# Module 3 Homework: Data Warehousing & BigQuery

## Loading the data

You can use the following scripts to load the data into your GCS bucket:

In [1]:
import os
import json
import tzdata, os

os.environ["BUCKET_URL"] = "gs://ny_taxi_aletbm"

with open(os.environ["GOOGLE_APPLICATION_CREDENTIALS"]) as f:
    os.environ["DESTINATION__CREDENTIALS"] = json.dumps(json.load(f))

In [4]:
def download_tzdata_windows(
    base_dir=None,
    year=2022,
    name="tzdata"
):
    import os
    import tarfile
    import urllib3

    http = urllib3.PoolManager()
    folder = base_dir if base_dir else os.path.join(os.path.expanduser('~'), "Downloads")
    tz_path = os.path.join(folder, "tzdata.tar.gz")
    
    with open(tz_path, "wb") as f:
        f.write(http.request('GET', f'https://data.iana.org/time-zones/releases/tzdata{year}f.tar.gz').data)
    
    folder = os.path.join(folder, name)
    
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    tarfile.open(tz_path).extractall(folder)
    
    with open(os.path.join(folder, "windowsZones.xml"), "wb") as f:
        f.write(http.request('GET', f'https://raw.githubusercontent.com/unicode-org/cldr/master/common/supplemental/windowsZones.xml').data)
download_tzdata_windows(year=2022)


C:\Users\Alexander\AppData\Local\Temp\ipykernel_11292\179648397.py:22: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tarfile.open(tz_path).extractall(folder)


In [5]:
import dlt
import requests
import pandas as pd
from dlt.destinations import filesystem
from io import BytesIO

Ingesting parquet files to GCS.

In [6]:
# Define a dlt source to download and process Parquet files as resources
@dlt.source(name="rides")
def download_parquet():
    prefix = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata"
    for month in range(1, 7):
        file_name = f"yellow_tripdata_2024-0{month}.parquet"
        url = f"{prefix}_2024-0{month}.parquet"
        response = requests.get(url)

        df = pd.read_parquet(BytesIO(response.content))

        # Return the dataframe as a dlt resource for ingestion
        yield dlt.resource(df, name=file_name)


# Initialize the pipeline
pipeline = dlt.pipeline(
    pipeline_name="rides_pipeline",
    destination=filesystem(layout="{schema_name}/{table_name}.{ext}"),
    dataset_name="rides_dataset",
)

# Run the pipeline to load Parquet data into DuckDB
load_info = pipeline.run(download_parquet(), loader_file_format="parquet")

# Print the results
print(load_info)

Pipeline rides_pipeline load step completed in 1 minute and 3.57 seconds
1 load package(s) were loaded to destination filesystem and into dataset rides_dataset
The filesystem destination used gs://ny_taxi_aletbm location to store data
Load package 1770651736.0392327 is LOADED and contains no failed jobs


## Question 1. Counting records

What is count of records for the 2024 Yellow Taxi Data?
- 65,623
- 840,402
- 20,332,093
- 85,431,289

### Querys

Create an schema:
```sql
CREATE SCHEMA `[PROJECT_ID].[DATABASE]`
OPTIONS (
  location = "US"
);
```

Create an external table using the Yellow Taxi Trip Records:
```sql
CREATE OR REPLACE EXTERNAL TABLE `[PROJECT_ID].[DATABASE].yellow_taxi_external`
OPTIONS (
  format = 'PARQUET',
  uris = [
    'gs://[BUCKET_NAME]/rides_dataset/rides/yellow_tripdata_2024_*.parquet'
  ]
);
```

Create a (regular/materialized) table in BQ using the Yellow Taxi Trip Records
```sql
CREATE OR REPLACE TABLE `[PROJECT_ID].[DATABASE].yellow_taxi_2024`
AS 
SELECT * FROM `[PROJECT_ID].[DATABASE].yellow_taxi_external`;
```

Count of records:
```sql
SELECT COUNT(*) FROM `[PROJECT_ID].[DATABASE].yellow_taxi_external`;
```

Anwser: 20,332,093

## Question 2. Data read estimation

Write a query to count the distinct number of PULocationIDs for the entire dataset on both the tables.
 
What is the **estimated amount** of data that will be read when this query is executed on the External Table and the Table?

- 18.82 MB for the External Table and 47.60 MB for the Materialized Table
- 0 MB for the External Table and 155.12 MB for the Materialized Table
- 2.14 GB for the External Table and 0MB for the Materialized Table
- 0 MB for the External Table and 0MB for the Materialized Table

### Querys

```sql
SELECT
  COUNT(DISTINCT pu_location_id) AS distinct_pu_locations
FROM `[PROJECT_ID].[DATABASE].yellow_taxi_external`;
```

```sql
SELECT
  COUNT(DISTINCT pu_location_id) AS distinct_pu_locations
FROM `[PROJECT_ID].[DATABASE].yellow_taxi_2024`;
```

Anwser: 0 MB for the External Table and 155.12 MB for the Materialized Table

## Question 3. Understanding columnar storage

Write a query to retrieve the PULocationID from the table (not the external table) in BigQuery. Now write a query to retrieve the PULocationID and DOLocationID on the same table.

Why are the estimated number of Bytes different?
- BigQuery is a columnar database, and it only scans the specific columns requested in the query. Querying two columns (PULocationID, DOLocationID) requires 
reading more data than querying one column (PULocationID), leading to a higher estimated number of bytes processed.
- BigQuery duplicates data across multiple storage partitions, so selecting two columns instead of one requires scanning the table twice, 
doubling the estimated bytes processed.
- BigQuery automatically caches the first queried column, so adding a second column increases processing time but does not affect the estimated bytes scanned.
- When selecting multiple columns, BigQuery performs an implicit join operation between them, increasing the estimated bytes processed

### Querys

Query to retrieve the PULocationID from the table:
```sql
SELECT pu_location_id
FROM `[PROJECT_ID].[DATABASE].yellow_taxi_2024`
LIMIT 10;
```

Query to retrieve the PULocationID and DOLocationID on the same table
```sql
SELECT pu_location_id, do_location_id
FROM `[PROJECT_ID].[DATABASE].yellow_taxi_2024`
LIMIT 10;
```
Anwser: BigQuery is a columnar database, and it only scans the specific columns requested in the query. Querying two columns (PULocationID, DOLocationID) requires 
reading more data than querying one column (PULocationID), leading to a higher estimated number of bytes processed.

## Question 4. Counting zero fare trips

How many records have a fare_amount of 0?
- 128,210
- 546,578
- 20,188,016
- 8,333

### Query

```sql
SELECT COUNT(*) AS zero_fare_count
FROM `[PROJECT_ID].[DATABASE].yellow_taxi_2024`
WHERE fare_amount = 0;
```

Anwser: 8,333

## Question 5. Partitioning and clustering

What is the best strategy to make an optimized table in Big Query if your query will always filter based on tpep_dropoff_datetime and order the results by VendorID (Create a new table with this strategy)

- Partition by tpep_dropoff_datetime and Cluster on VendorID
- Cluster on by tpep_dropoff_datetime and Cluster on VendorID
- Cluster on tpep_dropoff_datetime Partition by VendorID
- Partition by tpep_dropoff_datetime and Partition by VendorID

Anwser: Partition by tpep_dropoff_datetime and Cluster on VendorID

## Explanation
+ Partitioning:
    + BigQuery partitions are used to filter large datasets efficiently.
    + Since your queries always filter on tpep_dropoff_datetime, partitioning by this column reduces the number of bytes scanned, making queries faster and cheaper.

+ Clustering:
    + Clustering organizes the data within each partition based on the values of one or more columns.
    + Because your queries order results by VendorID, clustering on VendorID improves sorting and aggregation performance.

```sql
CREATE OR REPLACE TABLE `[PROJECT_ID].[DATABASE].yellow_taxi_optimized`
PARTITION BY DATE(tpep_dropoff_datetime)
CLUSTER BY vendor_id AS
SELECT *
FROM `[PROJECT_ID].[DATABASE].yellow_taxi_2024`;
```

## Question 6. Partition benefits

Write a query to retrieve the distinct VendorIDs between tpep_dropoff_datetime
2024-03-01 and 2024-03-15 (inclusive)

Use the materialized table you created earlier in your from clause and note the estimated bytes. Now change the table in the from clause to the partitioned table you created for question 5 and note the estimated bytes processed. What are these values? 

Choose the answer which most closely matches.
 
- 12.47 MB for non-partitioned table and 326.42 MB for the partitioned table
- 310.24 MB for non-partitioned table and 26.84 MB for the partitioned table
- 5.87 MB for non-partitioned table and 0 MB for the partitioned table
- 310.31 MB for non-partitioned table and 285.64 MB for the partitioned table

### Querys:

Query to retrieve the distinct VendorIDs between tpep_dropoff_datetime
2024-03-01 and 2024-03-15 on materialized table:
```sql
SELECT DISTINCT vendor_id
FROM `[PROJECT_ID].[DATABASE].ny_taxi.yellow_taxi_2024`
WHERE DATE(tpep_dropoff_datetime) BETWEEN '2024-03-01' AND '2024-03-15';
```

Query to retrieve the distinct VendorIDs between tpep_dropoff_datetime
2024-03-01 and 2024-03-15 on partitioned table:
```sql
SELECT DISTINCT vendor_id
FROM `[PROJECT_ID].[DATABASE].ny_taxi.yellow_taxi_optimized`
WHERE DATE(tpep_dropoff_datetime) BETWEEN '2024-03-01' AND '2024-03-15';
```

Anwser: 310.24 MB for non-partitioned table and 26.84 MB for the partitioned table

## Question 7. External table storage

Where is the data stored in the External Table you created?

- Big Query
- Container Registry
- GCP Bucket
- Big Table

Answer: GCP Bucket

### Explanation:

+ When you create an External Table in BigQuery, BigQuery does not copy the data into its own storage.
+ Instead, it references the data where it lives, in your case in Parquet files on GCS.
+ Queries on the External Table read the data directly from GCS each time they run.

## Question 8. Clustering best practices

It is best practice in Big Query to always cluster your data:
- True
- False

Clustering is best practice only when your queries benefit from it, not always.
Answer: False


## Question 9. Understanding table scans

No Points: Write a `SELECT count(*)` query FROM the materialized table you created. How many bytes does it estimate will be read? Why?

### Query

```sql
SELECT COUNT(*) FROM `poetic-genius-486912-k5.ny_taxi.yellow_taxi_2024`;
```

It shows 0 B because BigQuery returned the result from cached query results. To see the actual bytes processed, cached results must be disabled.
